In [ ]:
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

In [ ]:
scene = load_scene(filename="minis/row1_col4.xml")
scene.

In [3]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …